In [1]:
import requests
import re
import urllib 
from bs4 import BeautifulSoup
import string
import pandas as pd
import numpy as np

In [ ]:
temporary_links = []
primary_links = []
product_links = []
titles = []
categories=[]
prices = []
artics=[]
brands=[]

url = 'https://www.jonnesway.ru/catalog/'
r = requests.get(url)
html_content = r.text
soup = BeautifulSoup(html_content,'html.parser')
for link in soup.findAll('a', attrs={'href': re.compile("/catalog/")}):
    temporary_links.append(link.get('href'))
for link in np.unique(temporary_links):
    if  len(re.split(r'(\W+)', link)) >= 6 and any(map(str.isdigit, re.split(r'(\W+)', link)[4])):
        primary_links.append(link)
        
for primary_link in primary_links:
    url = 'https://www.jonnesway.ru' + primary_link
    r = requests.get(url)
    html_content = r.text
    soup = BeautifulSoup(html_content,'html.parser')
    temporary_links = []
    for link in soup.findAll('a', attrs={'href': re.compile("/catalog/")}):
        temporary_links.append(link.get('href'))
    for link in np.unique(temporary_links):
        if link not in primary_links and len(re.split(r'(\W+)', link)) >= 6 and any(map(str.isdigit, re.split(r'(\W+)', link)[4])):
            primary_links.append(link)
    for link in soup.findAll('a', attrs={'href': re.compile("/product/")}):
        if link not in product_links:
            product_links.append(link.get('href'))
product_links = np.unique(product_links)

for product_link in product_links:
    url = 'https://www.jonnesway.ru' + product_link
    r = requests.get(url)
    html_content = r.text
    soup = BeautifulSoup(html_content,'html.parser')
    
    title = soup.find_all('title')[0].text
    titles.append(title)
    
    tags = soup.find_all('tr')
    for tag in tags:
        if 'р.' in tag.get_text():
            price = tag.get_text()
    price = "".join(price.replace("р.","").split())
    prices.append(price)
    
    numb=str(soup.find_all('h1')).find('артикул')
    artic=str(soup.find_all('h1'))[numb+9:-6]
    artics.append(artic)
    
    brands.append('Jonnesway')
    
    n=soup.find_all('a')
    for i in range(len(n)):
        if n[i].text=='перейти к сравнению продуктов':
            category=n[i-1].text
    categories.append(category) 
    
product_links =[ 'https://www.jonnesway.ru%s'%x for x in product_links ]

table = pd.DataFrame(list(zip(titles,brands, artics,prices,categories, product_links)), columns =['Наименование товара', 'Бренд','Артикул','Цена, р.', 'Категория','Ссылка']) 

table.to_excel('jonnesway.xlsx')